In [3]:
import pandas as pd 
import numpy as np
admissions=pd.read_csv("d://python/sklearn/admissions.csv")
admissions["actual_label"]=admissions["admit"]
admissions=admissions.drop("admit",axis=1)

shuffled_index=np.random.permutation(admissions.index)
shuffled_admissions=admissions.loc[shuffled_index]
admissions=shuffled_admissions.reset_index()
admissions.ix[0:128,"fold"]=1
admissions.ix[129:257,"fold"]=2
admissions.ix[258:386,"fold"]=3
admissions.ix[387:514,"fold"]=4
admissions.ix[515:644,"fold"]=5
admissions["fold"]=admissions["fold"].astype('int')
print(admissions.head())
print(admissions.tail())

   index       gpa         gre  actual_label  fold
0    500  2.965277  665.958871             1     1
1    226  3.767447  565.906440             0     1
2    570  3.549322  645.261912             1     1
3    248  2.993313  494.688082             0     1
4    580  3.379220  590.787054             1     1
     index       gpa         gre  actual_label  fold
639      5  3.599108  442.763567             0     5
640    148  3.000161  533.211280             0     5
641     61  3.049961  602.001883             0     5
642    474  3.206377  759.037953             1     5
643    165  3.320750  716.173275             0     5


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  i

In [5]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
train_iteration_one=admissions[admissions["fold"]!=1]
test_iteration_one=admissions[admissions["fold"]==1]
model.fit(train_iteration_one[["gpa"]],train_iteration_one["actual_label"])

labels=model.predict(test_iteration_one[["gpa"]])
test_iteration_one["predicted_label"]=labels
matches=test_iteration_one["predicted_label"]==test_iteration_one["actual_label"]
correct_predictions=test_iteration_one[matches]
iteration_one_accuracy=len(correct_predictions)/float(len(test_iteration_one))
print(iteration_one_accuracy)

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6511627906976745


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
fold_ids=[1,2,3,4,5]
def train_and_test(df,folds):
    fold_accuracies=[]
    for fold in folds:
        model=LogisticRegression()
        train=admissions[admissions["fold"]!=fold]
        test=admissions[admissions["fold"]==fold]
        model.fit(train[["gpa"]],train["actual_label"])

        labels=model.predict(test[["gpa"]])
        test["predicted_label"]=labels
        
        matches=test["predicted_label"]==test["actual_label"]
        correct_predictions=test[matches]
        fold_accuracies.append(len(correct_predictions)/float(len(test)))
    return(fold_accuracies)
accuracies=train_and_test(admissions,fold_ids)
print(accuracies)

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

[0.6511627906976745, 0.5658914728682171, 0.6744186046511628, 0.7265625, 0.6201550387596899]


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
average_accuracy=np.mean(accuracies)
print(average_accuracy)

0.6476380813953488


In [11]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf=KFold(n_splits=5,shuffle=True,random_state=8)
lr=LogisticRegression()

accuracies=cross_val_score(lr,admissions[["gpa"]],admissions["actual_label"],scoring="accuracy",cv=kf)
average_accuracy=sum(accuracies)/len(accuracies)
print(accuracies)
print(average_accuracy)

[0.65116279 0.59689922 0.63565891 0.66666667 0.640625  ]
0.638202519379845


d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t